# GNOD 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# 2. find url and store it in a variable
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
# 3. download html with a get request
response = requests.get(url)
response.status_code

200

In [4]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
# 4.2. check that the html code looks like it should
# soup

In [6]:
# 5. retrieve/extract the desired info (here, you'll paste the "Selector" you copied before to get the element that belongs to the top movie)
soup.select("#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p")

[<p class="title-artist"><cite class="title">Last Night</cite><em class="artist">Morgan Wallen</em></p>]

In [7]:
# soup.select("div.chart-content.col-xs-12.col-sm-8 > p") # all the info about all the songs

In [8]:
# To get the title
soup.select("div.chart-content.col-xs-12.col-sm-8 > p > cite")[0].get_text()

'Last Night'

In [9]:
# To get the artist
soup.select("div.chart-content.col-xs-12.col-sm-8 > p > em")[0].get_text()

'Morgan Wallen'

In [10]:
#initialize empty lists
title = []
artist = []


# define the number of iterations of our for loop
# by checking how many elements are in the retrieved result set
# (this is equivalent but more robust than just explicitly defining 250 iterations)
num_iter = len(soup.select("div.chart-content.col-xs-12.col-sm-8 > p"))

tClist = soup.select("div.chart-content.col-xs-12.col-sm-8 > p > cite")
artistlist = soup.select("div.chart-content.col-xs-12.col-sm-8 > p > em")
# iterate through the result set and retrive all the data
for i in range(num_iter):
    title.append(tClist[i].get_text())
    artist.append(artistlist[i].get_text())

# print(title)
# print(artist)

In [11]:
# each list becomes a column
music = pd.DataFrame({"title":title,
                       "artist":artist
                      })

In [12]:
music

,title,artist
0,Last Night,Morgan Wallen
1,Flowers,Miley Cyrus
2,Fast Car,Luke Combs
3,Thank God I Do,Lauren Daigle
4,Like Crazy (Deep House Remix),Jimin
...,...,...
95,Shivers,Ed Sheeran
96,Tennessee Orange,Megan Moroney
97,God Is In This Story,Katy Nichole & Big Daddy Weave
98,What He Didn't Do,Carly Pearce


# GNOD 2

In [13]:
# 1st question
# artist_name = input (" What´s the singer´s name? ")

In [14]:
# 2nd question
# song_title = input ("What´s the song title? ")

In [15]:
music_title = music.sample(n=1)
# print (pd.Series(music_title['title'].values), pd.Series(music_title['artist'].values)) 

In [16]:
# if len(music[(music['artist'] == artist_name) & (music['title'] == song_title)]) == 1:
#     print (music_title)
# else:
#     print ("Sorry, we don´t have any recommendation for you")

# GNOD 3

In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep

**Storing secrets**

In [18]:
def storing_secrets(password):
    secrets_file = open(password,"r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            secrets_dict[line.split(' -> ')[0]]=line.split(' -> ')[1].strip()
    return secrets_dict

In [19]:
secrets_dict = storing_secrets("Spotify API.txt")

**Authentication with secrets**

In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['Client ID'],
                                                           client_secret=secrets_dict['Client Secret']))

In [21]:
playlist = sp.user_playlist_tracks("spotify", "3eSxghZaZ5XwsCGGC211O2")


In [22]:
# It is limited to 100
# playlist
playlist["total"]

399

In [23]:
display(playlist["items"][0]["track"]["name"]) # song
display(playlist["items"][0]["track"]["artists"][0]["name"]) # artist
display(playlist["items"][0]["track"]["uri"]) # uri
display(playlist["items"][0]["track"]['external_urls']['spotify']) # track url

'Diles'

'Bad Bunny'

'spotify:track:6C1RD7YQVvt3YQj0CmuTeu'

'https://open.spotify.com/track/6C1RD7YQVvt3YQj0CmuTeu'

In [24]:
# Create a function to get the song, artist, uri and track url.
def get_name_artists_from_track(track):
    return [(track["track"]["name"],artist["name"],track["track"]['external_urls']['spotify'],track["track"]['uri']) for artist in track["track"]["artists"]]

In [25]:
# For flatten them
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [30]:
# This function will show a Dataframe for the first 100 songs
def create_dataframe(playlist, sp):
        
    all_songs_artists = list(map(get_name_artists_from_track,playlist["items"]))
    
    data = pd.DataFrame(flatten(all_songs_artists), columns =['title', 'artists', 'uri','url'])
     
    audio_features =[]
    i = 0
    for item in data['uri']:
        audio_features.append(sp.audio_features(item)[0])
        if i % 5 == 0:
            sleep(randint(1,3000)/1000)
        i = i + 1
        
    audio_features = pd.DataFrame (audio_features)
        
    audio_features = audio_features.drop(['id','uri','track_href', 'type','analysis_url'], axis =1)
        
    All_data = pd.concat([data, audio_features], axis=1)
    All_data = All_data.drop ('uri', axis =1)
    All_data = All_data.drop_duplicates(subset=['url']).reset_index(drop=True)
    return All_data

In [31]:
create_dataframe(playlist, sp)

,title,artists,url,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Diles,Bad Bunny,spotify:track:6C1RD7YQVvt3YQj0CmuTeu,0.813,0.560,2,-5.904,0,0.0618,0.1080,0.000000,0.2680,0.287,118.033,286041,4
7,"Cayó La Noche (feat. Cruz Cafuné, Abhir Hathi,...",La Pantera,spotify:track:0Bv7O6fmZk3GrNQFYJaXPn,0.672,0.745,5,-5.269,0,0.3000,0.4380,0.000009,0.0699,0.739,173.974,413111,4
14,Limbo,Daddy Yankee,spotify:track:3pDhN3qB33AOPhQEkUCaWt,0.806,0.875,7,-3.817,1,0.0880,0.0838,0.001920,0.0650,0.917,125.063,224693,4
15,Lovumba,Daddy Yankee,spotify:track:6oyl7ELfrMqXLhGmIjXY9R,0.734,0.933,11,-4.639,0,0.0578,0.0375,0.000000,0.3020,0.935,120.987,218413,4
16,No Sigue Modas,Juan Magán,spotify:track:0Qqrehp0IC6ppqIUz6mPCe,0.709,0.801,0,-4.688,1,0.0533,0.0252,0.000000,0.1590,0.829,128.007,189840,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Ay Vamos,J Balvin,spotify:track:6Ges5C2IE738iJh4HyQizQ,0.718,0.711,8,-5.746,0,0.1100,0.1570,0.000055,0.1000,0.920,173.916,226413,4
242,6 AM,J Balvin,spotify:track:7mPwkbaoiQ5bf8FPbRJzUr,0.747,0.739,5,-5.256,0,0.0931,0.1420,0.000000,0.2730,0.737,175.978,243227,4
244,Ginza,J Balvin,spotify:track:2C2TGgFzrTRIOdQS1vUN5h,0.730,0.809,5,-6.406,0,0.0876,0.2080,0.001300,0.0804,0.825,101.965,171093,4
245,La Nueva Y La Ex,Daddy Yankee,spotify:track:6WuufZDBNoEJBTinRiznhO,0.640,0.943,5,-3.542,1,0.1370,0.0464,0.000000,0.1370,0.637,198.062,197053,4


In [34]:
def get_playlist(playlist_id):
    all_songs =[]
    all_songs.append (create_dataframe(playlist_id, sp))
    while playlist_id['next']!=None:
        playlist_id = sp.next(playlist_id)
        all_songs.append (create_dataframe(playlist_id, sp))
        print (len(all_songs))
    return all_songs

In [35]:
finalframe = get_playlist(playlist)

2
3
4


In [36]:
final = pd.concat (finalframe)

In [37]:
final

,title,artists,url,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Diles,Bad Bunny,spotify:track:6C1RD7YQVvt3YQj0CmuTeu,0.813,0.560,2,-5.904,0,0.0618,0.1080,0.000000,0.2680,0.287,118.033,286041,4
7,"Cayó La Noche (feat. Cruz Cafuné, Abhir Hathi,...",La Pantera,spotify:track:0Bv7O6fmZk3GrNQFYJaXPn,0.672,0.745,5,-5.269,0,0.3000,0.4380,0.000009,0.0699,0.739,173.974,413111,4
14,Limbo,Daddy Yankee,spotify:track:3pDhN3qB33AOPhQEkUCaWt,0.806,0.875,7,-3.817,1,0.0880,0.0838,0.001920,0.0650,0.917,125.063,224693,4
15,Lovumba,Daddy Yankee,spotify:track:6oyl7ELfrMqXLhGmIjXY9R,0.734,0.933,11,-4.639,0,0.0578,0.0375,0.000000,0.3020,0.935,120.987,218413,4
16,No Sigue Modas,Juan Magán,spotify:track:0Qqrehp0IC6ppqIUz6mPCe,0.709,0.801,0,-4.688,1,0.0533,0.0252,0.000000,0.1590,0.829,128.007,189840,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Tiempos de Plan B,Maldy,spotify:track:3ncDEAUzMdf291D27RJQdM,0.766,0.913,2,-3.760,1,0.0954,0.0879,0.000000,0.0622,0.720,167.927,161288,4
179,REMIX EXCLUSIVO,Feid,spotify:track:3eqCJfgJJs8iKx49KO12s3,0.651,0.557,5,-5.466,0,0.2040,0.0358,0.000000,0.1540,0.711,87.063,206947,4
180,Coco Chanel,Eladio Carrion,spotify:track:0PB0O24JqAuNdOAFVJljMS,0.683,0.764,2,-5.995,1,0.0441,0.0427,0.000000,0.0963,0.137,149.949,160325,4
182,Las Bratz (Remix),Aissa,spotify:track:36W1V6ft4fX7HQVSdIYL0J,0.810,0.815,10,-3.101,0,0.1000,0.2310,0.000000,0.0577,0.958,95.003,444632,4


In [38]:
final.to_csv('spotify_data.csv')

# Another playlist